In [1]:
import pandas as pd
import os
os.chdir('C:\\Users\\Tiziano Pacifico\\Desktop\\TESI')
import re

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

import nltk
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker
import csv

[nltk_data] Downloading package stopwords to C:\Users\Tiziano
[nltk_data]     Pacifico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Carico tutti i dataset in una lista di dataset
start = 1
end = 75
dfs = []
for i in range(start,end+1):
    read_file = f'redditDS\\splitted_df_sub_blob\\splitted_df_sub_blob{i}.json'
    df = pd.read_json(read_file, lines = True)
    dfs.append(df)

In [3]:
#ho bisogno di estrarre soltanto la colonna dei blob per tutti i df
blob_lists = []
for df in dfs:
    blob_list = list(df['blob'])
    blob_lists.append(blob_list)

In [4]:
# Carico i dizionari

In [5]:
base_path = 'FASE 2 Pulizia Testo\\Dizionari\\dict\\'
file = open(base_path + "acronimi.csv",'r')
acronimi = list(csv.reader(file, delimiter=","))[0]
file.close()

In [6]:
file = open(base_path + "keywords.csv",'r')
keywords = list(csv.reader(file, delimiter=","))[0]
file.close()

In [7]:
file = open(base_path + "persone.csv",'r')
persone = list(csv.reader(file, delimiter=","))[0]
file.close()

In [8]:
file = open(base_path + "hashtag.csv",'r')
hashtags = list(csv.reader(file, delimiter=","))[0]
file.close()

In [9]:
# Case normalization
# remove punctuations, emoji's, url, mentions
# Remove stopwords
# Stemming
# Remove numbers and word with numbers
# Accorpare le espressioni e i nomi propri
# Remove unknown words

In [10]:
stop = stopwords.words('english')
stemmer = PorterStemmer()
spell = SpellChecker()
blob_lists_1 = []
for bl in blob_lists:
    blob_1 = []
    for blob in bl:
        blob = blob.lower()
        blob = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", blob)
        blob = " ".join([word for word in blob.split() if word not in (stop)])
        blob = " ".join([stemmer.stem(word) for word in blob.split()])
        blob = ' '.join(s for s in blob.split() if not any(c.isdigit() for c in s))
        for k in keywords:
            if k in blob:
                #print(k)
                blob = blob.replace(k, k.replace(" ",""))
        for p in persone:
            if p in blob:
                #print(p)
                blob = blob.replace(p, p.replace(" ",""))
        for h in hashtags:
            if (h in blob and len(h.split())>1):
                blob = blob.replace(h, h.replace(" ",""))
        blob_1.append(blob)
    blob_lists_1.append(blob_1)

In [11]:
#creo dizionari con tutte le espressioni attaccate in una sola parola
keywords_nos = []
persone_nos = []
hashtags_nos = []
for k in keywords:
    keywords_nos.append(k.replace(" ",""))
for p in persone:
    persone_nos.append(p.replace(" ",""))
for h in hashtags:
    hashtags_nos.append(h.replace(" ",""))

In [12]:
#Creo un'unica lista per tutti i dizionari
all_dict = []
all_dict.extend(acronimi)
all_dict.extend(keywords_nos)
all_dict.extend(persone_nos)
all_dict.extend(hashtags_nos)

In [13]:
#Controllo sule parole sconosciute
blob_list_2 = []
for bl in blob_lists_1:
    blob_1 = []
    for blob in bl:
        sconosciute = list(spell.unknown([w for w in blob.split()]))
        res = list(filter(lambda x: x not in set(all_dict),sconosciute))
        blob = ' '.join(word for word in blob.split() if word not in res)
        blob_1.append(blob)
    blob_list_2.append(blob_1)

In [14]:
dfs_1 = []
for i,df in enumerate(dfs):
    df['clean_blob'] = blob_list_2[i]
    df = df.drop('blob', axis=1)
    dfs_1.append(df)
    df.to_json(f"RedditDS\\splitted_df_cleaned_a\\splitted_df_cleaned{i}_a.json",orient='records',lines=True)